# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
ratings2 = ratings.groupby('userId').agg({'timestamp': ['max', 'min','count']})

In [8]:
def life_cycle(user):
    return (int(user['timestamp']['max']) - int(user['timestamp']['min']))

In [9]:
ratings2['life_cycle'] = ratings2.apply(life_cycle, axis = 1)
ratings2[ratings2['timestamp']['count'] > 100]

timestamp                   life_cycle
               max         min count           
userId                                         
4        949982274   949778714   204     203560
8       1154474527  1154389340   116      85187
15      1469330735   997937239  1700  471393496
17      1127476640  1127468587   363       8053
19       855195373   855190091   423       5282
...            ...         ...   ...        ...
656      986244044   986240991   128       3053
659      866207451   834598040   142   31609411
664     1441911722  1343731283   519   98180439
665     1046967769   992836298   434   54131471
671     1074784735  1063500751   115   11283984

[258 rows x 4 columns]

In [25]:

print('Среднее время жизни пользоватлей поставиших  больше 100 оценок = ', int(ratings2['life_cycle'].mean()/(60*60*24))
      ,'день')

Среднее время жизни пользоватлей поставиших  больше 100 оценок =  201 день


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

# К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [26]:
import numpy as np

In [27]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [28]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [29]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [30]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [31]:
combined = rzd.merge(auto, how='outer').merge(air, how='outer')
combined

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [32]:
combined_adress = client_base.merge(combined, how='left', on='client_id')
combined_adress

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [33]:

#Вопрос. Я пытался добавить итоговые значения к таблице через pivot table. 
#Но что то пошло не так. Итоговые значения неправильно считаются. Как это поправить?

combined.pivot_table(index = 'client_id', values = ['rzd_revenue','auto_revenue','air_revenue'], 
                     aggfunc = np.sum, margins = True)

,air_revenue,auto_revenue,rzd_revenue
client_id,,,
111,0.0,0.0,1093.0
112,0.0,0.0,2810.0
113,0.0,57483.0,10283.0
114,0.0,83.0,5774.0
115,81.0,912.0,981.0
116,4.0,4834.0,0.0
117,13.0,98.0,0.0
118,173.0,0.0,0.0
All,81.0,912.0,981.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

# Ответ

Я думаю, координаты можно использовать для рекомендательных систем. 
Для связывания с визитом и фактом покупки, можно привязывать координаты местоположения покупок пользователей через tuple. 
Не уверен, что можно tuple добавить в дата фрейм, но что то в тьюпл, или список и двух элементов( долготы и широты). 
и следовательно к каждой покупке привязывать этот tuple.

Если координаты одного и того же пользователя значительно удаленные друг от друга, в нашей системе может означать, 
что он переехал в другой район, город. и т.д. или делает покупки дома утром и вечером, а днем на работе.
по всяким интернет покупкам, вообще будут другие координаты, поэтому интернет покупки надо будет исключать из совокупности.

Координаты на мой взгляд это что то типо векторов (которые мы изучали в прошлых лекциях) если много схожих координат, можно строить закономерности в покупках пользователей, и рекомендовать им похожие магазины с интересующих их товарами
или привязать эти координаты к яндекс картам например.

Какой правильный ответ у этой задачи:?=)